In [39]:
import torch
# ! pip install numpy==2.0
print(torch.__version__)

2.2.2


In [40]:
from torch import nn
import torch.nn.functional as F
# multi classification [0.3, 0.2, 0.1, 0.4]

class MultiClassificationModel(nn.Module):
  def __init__(self):
    super(MultiClassificationModel, self).__init__()
    # 13 because we dropped Student ID as a feature
    self.ly1 = nn.Linear(8, 30)
    self.dp1 = nn.Dropout(p=0.5)
    self.ly2 = nn.Linear(30, 20)
    self.dp2 = nn.Dropout(p=0.5)
    self.ly3 = nn.Linear(20,15)
    self.dp3 = nn.Dropout(p=0.5)
    self.ly4 = nn.Linear(15, 10)
    self.dp4 = nn.Dropout(p=0.5)
    self.ly5 = nn.Linear(10, 5)
    self.act = nn.Softmax(dim=1)

  def forward(self, x):
    x = self.dp1(F.relu(self.ly1(x)))
    x = self.dp2(F.relu(self.ly2(x)))
    x = self.dp3(F.relu(self.ly3(x)))
    x = self.dp4(F.relu(self.ly4(x)))
    x = self.act(self.ly5(x))
    return x
  
model = MultiClassificationModel()

In [41]:
import torch
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset

df = pd.read_csv('star_classification.csv')

# check for nulls
# print(df.isnull().sum())

# get labels
labels = LabelEncoder().fit_transform(df['class'])
# drop label and drop irrelevant data so it doesn't pollute the feature data
features = df.drop(columns=['obj_ID', 'run_ID', 'rerun_ID', 'cam_col', 'field_ID', 'spec_obj_ID', 'class', 'plate', 'MJD', 'fiber_ID'])
# print(features)
# Normalize data
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)
# print(features)

# Create training, evaluation data groups
training_f, testing_f, training_l, testing_l = train_test_split(scaled_features, labels, test_size=0.2, random_state=42)

# Turn it all into tensors & make ready to load into Pytorch
training_f = torch.tensor(training_f, dtype=torch.float32)
testing_f = torch.tensor(testing_f, dtype=torch.float32)
# For multi classification our label data needs to be long (int) not float
training_l = torch.tensor(training_l, dtype=torch.long)
testing_l = torch.tensor(testing_l, dtype=torch.long)
# print(testing_l)

# print(training_f)
# print(testing_f)
# print(training_l)
# print(testing_l)

training_dataset = TensorDataset(training_f, training_l)
testing_dataset = TensorDataset(testing_f, testing_l)
training_loader = DataLoader(training_dataset, batch_size=32, shuffle=True)
testing_loader = DataLoader(testing_dataset, batch_size=32, shuffle=True)


In [42]:
from torch import optim

# Make loss function and optimizer
criterion = nn.CrossEntropyLoss() # Using different loss function for Multi Classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0
    model.train()
    for features, labels in training_loader:
        optimizer.zero_grad()
        output = model(features) # model.forward()
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() # keep track of overall loss rate
    avg_loss = running_loss/len(training_loader)
    if not(epoch+1) % 2:
        print(f"Average Loss {avg_loss}, Epoch: {epoch+1}")

Average Loss 1.1661778667449951, Epoch: 2
Average Loss 1.1592050133705138, Epoch: 4
Average Loss 1.158071209859848, Epoch: 6
Average Loss 1.156832037472725, Epoch: 8
Average Loss 1.1550382762432099, Epoch: 10


In [43]:
# Evaluation
from torchmetrics import Accuracy

acc = Accuracy(task='multiclass', num_classes=5) # [A,B,C,D,F] -> [0,1,2,3,4]
model.eval()

with torch.no_grad():
    for features, labels in testing_loader:
        output = model(features)
        # multi: We need to get the highest probability from all [0.2,0.3,0.5,0.1] the probabilities for EACH possible class/label
        _, predicted = torch.max(output.data, 1) 
        acc.update(predicted, labels)

print(f"Accuracy: {acc.compute().item()}") # higher is better, 0.87 represents 87% accuracy

Accuracy: 0.7549499869346619
